In [1]:
import numpy as np
import csv

from Classifiers.Bayes_Parzen import *
from Classifiers.k_means_clustering import *
from Classifiers.k_nearest_neighbour import *
from Classifiers.logistic import *
from Classifiers.Naive_Bayes_bins import *
from Classifiers.solver import *
from Classifiers.compute_performance import *

In [2]:
data = np.array(list(csv.reader(open("DataSets/Hepatitis/hepatitis.csv"), delimiter=',')))
N = data.shape[0]
X, y = data[:, 1:], data[:, 0].astype(np.int)
X[X == "?"] = "0"
X = X.astype(np.float)
y = y - 1
D, num_classes = X.shape[1], 2
for i in range(D):
    if i == 13 or i == 16:
        X[X[:, i] == 0, i] = np.mean(X[X[:, i] != 0, i])
    else:
        X[X[:, i] == 0, i] = round(np.mean(X[X[:, i] != 0, i]))
    X[:, i] = (X[:, i] - np.min(X[:, i])) / (np.max(X[:, i]) - np.min(X[:, i]))
X = np.delete(X, (17), axis=1)
D, num_classes = X.shape[1], 2
data = {}
random_perm = np.random.permutation(N)
data['X_train'] = X[random_perm[:115]]
data['y_train'] = y[random_perm[:115]]
data['X_test'] = X[random_perm[115:]]
data['y_test'] = y[random_perm[115:]]
for k, v in list(data.items()):
    print(('%s: ' % k, v.shape))
print(D, num_classes)

('X_train: ', (115, 18))
('y_train: ', (115,))
('X_test: ', (40, 18))
('y_test: ', (40,))
18 2


In [3]:
naive_bayes = Naive_Bayes(D, num_classes, 5)
naive_bayes.train(data['X_train'], data['y_train'])
y_pred1 = naive_bayes.test(data['X_test'])
performance = compute_performance(y_pred1, data['y_test'], num_classes)
for k, v in list(performance.items()):
    print(('%s: ' % k, v))

('accuracy: ', 0.85)
('precision: ', array([1.  , 0.85]))
('recall: ', array([0., 1.]))
('F1: ', 0.45945945945945943)


In [4]:
bayes = Bayes_Parzen(num_classes)
bayes.train(data['X_train'], data['y_train'])
y_pred2 = bayes.test(data['X_test'], h=1)
performance = compute_performance(y_pred2, data['y_test'], num_classes)
for k, v in list(performance.items()):
    print(('%s: ' % k, v))

('accuracy: ', 0.85)
('precision: ', array([1.  , 0.85]))
('recall: ', array([0., 1.]))
('F1: ', 0.45945945945945943)


In [5]:
data['X_train'] = X[random_perm[:80]]
data['y_train'] = y[random_perm[:80]]
data['X_val'] = X[random_perm[80:115]]
data['y_val'] = y[random_perm[80:115]]

In [6]:
knn = KNearestNeighbor()
knn.train(data['X_train'], data['y_train'])
k_choices = [3, 5, 7, 9, 11]
best_k, best_f1 = 1, 0.0
for k in k_choices:
    y_pred3 = knn.test(data['X_val'], k=k)
    per = compute_performance(y_pred3, data['y_val'], num_classes)
    if per['F1'] > best_f1:
        best_k = k
        best_f1 = per['F1']
y_pred3 = knn.test(data['X_test'], k=best_k)
performance = compute_performance(y_pred3, data['y_test'], num_classes)
for k, v in list(performance.items()):
    print(('%s: ' % k, v))

('accuracy: ', 0.9)
('precision: ', array([0.75      , 0.91666667]))
('recall: ', array([0.5       , 0.97058824]))
('F1: ', 0.7714285714285714)


In [7]:
K_choices = [2, 3, 5, 7, 8, 9]
best_k_means, best_f1 = None, 0.0
for K in K_choices:
    k_means = K_means(D, K=K)
    k_means.train(data['X_train'], data['y_train'], num_iters=20)
    y_pred4 = k_means.test(data['X_val'])
    per = compute_performance(y_pred4, data['y_val'], num_classes)
    if per['F1'] > best_f1:
        best_k_means = k_means
        best_f1 = per['F1']
y_pred4 = best_k_means.test(data['X_test'])
performance = compute_performance(y_pred4, data['y_test'], num_classes)
for k, v in list(performance.items()):
    print(('%s: ' % k, v))

('accuracy: ', 0.9)
('precision: ', array([0.625  , 0.96875]))
('recall: ', array([0.83333333, 0.91176471]))
('F1: ', 0.8268398268398269)


In [8]:
logistic = Logistic(input_dim=D, num_classes=num_classes, weight_scale=1e-2)
solver = Solver(logistic, data, lr=2e-4, num_epochs=20, verbose=0)
solver.train()
y_pred5 = logistic.test(data['X_test'])
performance = compute_performance(y_pred5, data['y_test'], num_classes)
for k, v in list(performance.items()):
    print(('%s: ' % k, v))

('accuracy: ', 0.825)
('precision: ', array([0.        , 0.84615385]))
('recall: ', array([0.        , 0.97058824]))
('F1: ', 0.4520547945205479)
